In [ ]:
import pandas as pd

## Check HR

In [ ]:
path_to_folder = 'D:/DATA_THESIS/Projekt_nw_AG_AJ/Outputs_project_data_preprocessing'

In [ ]:
import os
import json
from tqdm import tqdm

# aggregated_data: data was generated using script data_preprocessing/samsung_polar_data_preprocessing.ipynb

def import_files_to_df(file_name, path = 'D:/DATA_THESIS/Projekt_nw_AG_AJ/dataset/aggregated_data'):
    
    data_all = pd.DataFrame()
    list_iter = os.listdir(path)
    list_of_iterations = []
    
    for iter in list_iter:
        list_of_data_for_all_patients = []
        list_id_patient = os.listdir(f'{path}/{iter}')
        for patient in tqdm(list_id_patient):
            path_to_file = f'{path}/{iter}/{patient}/{file_name}.csv'
            try:
                data = pd.read_csv(path_to_file)
                data['patient_id'] = patient
                data_all = pd.concat([data_all, data], axis = 0)
            except:
                print(path_to_file)
    return data_all

In [ ]:
def import_files_to_df_1_iter(file_name, iter,  path = 'D:/DATA_THESIS/Projekt_nw_AG_AJ/dataset/aggregated_data'):
    
        data_all = pd.DataFrame()
        list_of_iterations = []
        list_of_data_for_all_patients = []
        list_id_patient = os.listdir(f'{path}/{iter}')
        for patient in tqdm(list_id_patient):
            path_to_file = f'{path}/{iter}/{patient}/{file_name}.csv'
            try:
                data = pd.read_csv(path_to_file)
                data['patient_id'] = patient
                data_all = pd.concat([data_all, data], axis = 0)
            except:
                print(path_to_file)
        return data_all

In [ ]:
df_pedometer = import_files_to_df(file_name = 'minute_pedometer')

In [ ]:
len(df_pedometer['patient_id'].unique())

In [ ]:
df_pedometer[df_pedometer['patient_id'] == df_pedometer['patient_id'].unique()[1]]

In [ ]:
df_pedometer['ts'] = pd.to_datetime(df_pedometer['ts'])
df_pedometer['date'] = df_pedometer['ts'].dt.date
df_pedometer['hour'] = df_pedometer['ts'].dt.hour
df_pedometer

## PLOT

In [ ]:
import plotly.graph_objects as go

df = df_pedometer[df_pedometer['patient_id'] == df_pedometer['patient_id'].unique()[1]]

grouped = df.groupby('hour')


# Create traces for each day
traces = []
for day, group in grouped:
    trace = go.Bar(
        x=group['hour'],  # Hours
        y=group['running_steps'],  # Running steps
        name= f"{df['date'][day]}",
    )
    traces.append(trace)

# Create layout
layout = go.Layout(
    title='Running Steps Throughout the Day',
    xaxis=dict(title='Hour'),
    yaxis=dict(title='Number of Steps'),
    barmode='group',
)

# Create figure
fig = go.Figure(data=traces, layout=layout)

# Show plot
fig.show()

In [ ]:
df_pedometer[df_pedometer['patient_id'] == df_pedometer['patient_id'].unique()[5]]

In [ ]:
import plotly.express as px
import pandas as pd
df = df_pedometer[df_pedometer['patient_id'] == df_pedometer['patient_id'].unique()[1]]

df_grouped = df.groupby(["date", "hour"])['steps'].sum().reset_index()

# Create the bar plot using Plotly Express
fig = px.bar(df_grouped, x="hour", y="steps", color="date",
             title="Number of Steps Throughout the Day",
             labels={"hour": "Hour", "steps": "Number of Steps", "date": "Date"},
             barmode="group")

# Show the plot
fig.show()

## Check 3h before sleep

In [ ]:
# with open('m_e_emo_pre_loc_sleepSegments.pkl', 'rb') as f:
#       dataset = pd.read_pickle(f)

dataset = pd.read_pickle(f'{path_to_folder}/6_emotions_aggregates/m_e_emo_pre_loc_sleepSegments.pkl')

In [ ]:
dataset

In [ ]:
dataset[dataset['patient_id'] == df_pedometer['patient_id'].unique()[1]]

In [ ]:
df = df_pedometer[df_pedometer['patient_id'] == df_pedometer['patient_id'].unique()[1]].reset_index()

# Filter data for the same patient in dataset and convert 'filledTimestamp_evening' to datetime
df_data = dataset[dataset['patient_id'] == df_pedometer['patient_id'].unique()[1]].reset_index()

def count_steps_before_sleep(col_timestamp_before_sleep = 'filledTimestamp_evening', 
                             col_timestamp_steps = 'ts' ,  
                             dataframe_with_steps = df, 
                             dataframe_dataset = dataset ):

    """
    col_timestamp_before_sleep [str]: column with the end of the time frame we are interested in
    col_timestamp_steps [str]: column with the timestamp in the dataframe with steps
    dataframe_with_steps [DataFrame]: dataframe with steps
    dataframe_dataset [DataFrame]: dataframe with all the data for each day for each user / or dataframe with dates of sleep
    """

    dataframe_dataset[col_timestamp_before_sleep] = pd.to_datetime(dataframe_dataset[col_timestamp_before_sleep])
    dataframe_dataset['time_frame'] = dataframe_dataset[col_timestamp_before_sleep] - pd.Timedelta(hours=3)
    steps_3h_before_sleep = []
    patient_id_list = []
    dates_list = []
    for patient in (dataframe_dataset['patient_id']).unique():
        
        dataframe_dataset_patient = dataframe_dataset[dataframe_dataset['patient_id'] == patient]
        dataframe_with_steps_patient = dataframe_with_steps[dataframe_with_steps['patient_id'] == patient]
        
        
        for day in range(0, len(dataframe_dataset_patient['time_frame'])):
            list_of_steps_one_evening = []
            for i in range(0, len(dataframe_with_steps_patient[col_timestamp_steps])):
                if ((dataframe_with_steps_patient[col_timestamp_steps].iloc[i] > dataframe_dataset_patient['time_frame'].iloc[day]) 
                and (dataframe_with_steps_patient[col_timestamp_steps].iloc[i] < dataframe_dataset_patient['filledTimestamp_evening'].iloc[day])):
                    
                    list_of_steps_one_evening.append(dataframe_with_steps_patient['steps'].iloc[i])
            if list_of_steps_one_evening:
                num_steps_evening = max(list_of_steps_one_evening) - min(list_of_steps_one_evening)           
            else:
                num_steps_evening = 0 
            steps_3h_before_sleep.append(num_steps_evening)
            patient_id_list.append(patient)
            dates_list.append(dataframe_dataset_patient['dateOfSleep'].iloc[day])
    df_results = pd.DataFrame()
    df_results['steps_3h_before_sleep'] = steps_3h_before_sleep
    df_results['patient_id'] = patient_id_list
    df_results['dateOfSleep'] = dates_list
    return df_results

In [ ]:
df_with_steps = count_steps_before_sleep(col_timestamp_before_sleep = 'startTimestamp', 
                             col_timestamp_steps = 'ts' ,  
                             dataframe_with_steps = df_pedometer, 
                             dataframe_dataset = dataset )

In [ ]:
merged_df = pd.merge(df_with_steps, dataset, on=['dateOfSleep', 'patient_id'], how='inner')

In [ ]:
merged_df.columns

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Select the three columns for which you want to calculate the correlation
columns = ['start_hour', 'steps_3h_before_sleep', 'sleepQuality']

# Calculate the correlation matrix
correlation_matrix = merged_df[columns].corr()

# Plot the heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Heatmap of Three Columns')
plt.show()

## Count steps - all day

In [ ]:
df_pedometer

In [ ]:
# 1. Convert the 'timestamp' column to datetime type if it's not already
df_pedometer['ts'] = pd.to_datetime(df_pedometer['ts'])

# 2. Extract the date from the timestamp to create a new column representing the date
df_pedometer['date'] = df_pedometer['ts'].dt.date

# 3. Group the DataFrame by both 'id' and 'date'
grouped_df = df_pedometer.groupby(['patient_id', 'date'])

# 4. Calculate the maximum steps for each group
max_steps_per_id_per_date = (grouped_df['steps'].max() - grouped_df['steps'].min()).reset_index()
patient_list = []
dates_list = []
steps_count = []
# 5. Filter groups where min steps are not zero and print patient ID and date
for index, row in max_steps_per_id_per_date.iterrows():
    if grouped_df.get_group((row['patient_id'], row['date']))['steps'].min() != 0:
        print(f"Patient ID: {row['patient_id']}, Date: {row['date']}, {grouped_df.get_group((row['patient_id'], row['date']))['steps'].min()}")
        patient_list.append(row['patient_id'])
        dates_list.append(row['date'])
        steps_count.append(grouped_df.get_group((row['patient_id'], row['date']))['steps'].min())

In [ ]:
steps_not_0 = pd.DataFrame()
steps_not_0['patient_id'] = patient_list
steps_not_0['date'] = dates_list
steps_not_0['steps'] = steps_count
steps_not_0

In [ ]:
steps_not_0.to_csv(f'{path_to_folder}/6_emotions_aggregates/not_zero_steps_start.csv', index = False)

In [ ]:
len(steps_not_0['patient_id'].unique())

In [ ]:
grouped_df['steps'].min()

In [ ]:
grouped_df['steps'].max()

In [ ]:
max_steps_per_id_per_date['dateOfSleep'] = pd.to_datetime(max_steps_per_id_per_date['date']) + pd.DateOffset(days=1)
max_steps_per_id_per_date = max_steps_per_id_per_date.drop('date', axis = 1)

In [ ]:
# delete step values == 0, since that means there is no data for that day 
max_steps_per_id_per_date = max_steps_per_id_per_date[max_steps_per_id_per_date['steps'] != 0]
max_steps_per_id_per_date

In [ ]:
max_steps_per_id_per_date['dateOfSleep'] = pd.to_datetime(max_steps_per_id_per_date['dateOfSleep'])
dataset['dateOfSleep'] = pd.to_datetime(dataset['dateOfSleep'])
merged_all_days = pd.merge(max_steps_per_id_per_date, dataset,on=['dateOfSleep', 'patient_id'], how='inner')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Select the three columns for which you want to calculate the correlation
columns = ['steps', 'sleepQuality']

# Calculate the correlation matrix
correlation_matrix = merged_all_days[columns].corr()

# Plot the heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Heatmap of Three Columns')
plt.show()

In [ ]:
merged_all_days[['steps', 'dateOfSleep', 'patient_id', 'sleepQuality','stress']]

In [ ]:

# merged_all_days['sleepQ_stress'] = merged_all_days['sleepQuality']+ merged_all_days['stress']

# merged_all_days

In [ ]:
# merged_all_days.drop(['count_location_y'], axis = 1)
# merged_all_days['count_location'] = merged_all_days['count_location_x']
# merged_all_days.drop(['count_location_x'], axis = 1)
# merged_all_days.drop(['sleepQ_stress'], axis = 1)


In [ ]:
merged_all_days.to_pickle(f'{path_to_folder}/6_emotions_aggregates/m_e_emo_pre_loc_sleepSegments_steps.pkl')